In [1]:
import initialize_notebook # noqa

from hslu.dlm03.common import agent, backend, chat, chat_display, tools, types
from hslu.dlm03.tools import lint
from hslu.dlm03.util import ipython_utils, ratelimit, unified_diff

In [2]:
import pathlib
DOCKER_COMPOSE_FILE_PATH = pathlib.Path("/Users/vincent/Development/HSLU-DLM03-Devops-LLMs/docker-compose.yaml")
DOCKER_COMPOSE_FILE_PATH.write_text("")

0

In [3]:
from mcp.server import FastMCP
from pydantic import dataclasses
import docker
import secrets
import time
import yaml
import subprocess

SERVER = FastMCP()
DOCKER_CLIENT = docker.from_env()

@SERVER.tool()
def password_generator(length: int):
    return secrets.token_urlsafe(length)

@SERVER.tool()
def docker_list_images():
    return DOCKER_CLIENT.images.list()

@SERVER.tool()
def docker_list_containers():
    return DOCKER_CLIENT.containers.list()

@SERVER.tool()
def docker_compose_up():
    process = subprocess.run(
        [f"docker compose -f \"{DOCKER_COMPOSE_FILE_PATH}\" up --detach"],
        stdout=subprocess.PIPE, shell=True,
        check=False, stderr=subprocess.STDOUT, text=True,
    )
    return process.stdout

@SERVER.tool()
def edit_docker_compose(key: str, value: str | list[str]):
    """Sets the value corresponding to the key (given in fully qualified name, i.e. foo.bar.baz) in the Docker Compose File."""
    root = yaml.load(DOCKER_COMPOSE_FILE_PATH.read_text(), yaml.SafeLoader)
    if root is None:
        root = {}
    keys = key.split(".")
    data = root
    for key in keys[:-1]:
        if not key in data:
            data[key] = {}
        data = data[key]
    data[keys[-1]] = value
    DOCKER_COMPOSE_FILE_PATH.write_text(yaml.dump(root))
    return "Done"
            

@SERVER.tool()
def docker_logs(container_id: str):
    container = DOCKER_CLIENT.containers.get(container_id)
    return container.logs()

@SERVER.tool()
def wait(seconds: float):
    time.sleep(seconds)
    return "Done"

In [4]:
import threading

import uvicorn

PORT = 5000
HOST = "localhost"

RUN_ARGS = {
    "app": SERVER.streamable_http_app,
    "port": PORT,
    "host": HOST,
}

MCP_THREAD = threading.Thread(target=uvicorn.run, kwargs=RUN_ARGS)
MCP_THREAD.start()

In [5]:
BACKEND = backend.Gemini2p5Flash().get_async_backend()

In [6]:
MCP_SERVER_URL = f"http://{HOST}:{PORT}/mcp"
TOOL_MANAGER = tools.ToolManager.from_url(MCP_SERVER_URL)

In [7]:
SYSTEM_INSTRUCTIONS = """You are an expert AI Agent, specializing in deploying software applications using Docker.
Your task is to use the tools available to you to fulfill the user's request. You should not ask the user any more information, try to figure out on your own.
"""

In [8]:
AGENT = agent.Agent(BACKEND, TOOL_MANAGER)

In [9]:
CHAT = chat.Chat(messages=[{"role": "system", "content": SYSTEM_INSTRUCTIONS}])
ipython_utils.display_agent(AGENT, chat=CHAT)

HTML(value='<style>\n.chat-container {\n    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto…

In [10]:
# Can you deploy a MySQL Database?